Webscraping


In [86]:
from IPython.display import display
import json
import pandas as pd
pd.options.display.max_columns = None # Display all columns of a dataframe
pd.options.display.max_rows = 700
from pprint import pprint
import re
import requests
from bs4 import BeautifulSoup
#test coucouc

In [87]:
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

chromeOptions = webdriver.ChromeOptions()
#it is way to add options

path = "C:/Users/User/Documents/GitHub/Films_DM_2023/chromedriver_win32" #the path
driver = webdriver.Chrome(executable_path=os.path.join(path, 'chromedriver'))

driver.get("https://www.allocine.fr/")
time.sleep(1) # Let the user actually see something!

C:\Users\User\AppData\Local\Temp\ipykernel_28192\1295019404.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=os.path.join(path, 'chromedriver'))


In [88]:
#Accepter les cookies pour pouvoir passer à la suite
cookies = driver.find_element(By.CLASS_NAME, "jad_cmp_paywall_button-cookies").click()
time.sleep(2)


In [89]:
#Aller dans films 
films = driver.find_element(By.XPATH, '//*[@title="Cinéma"]').click()
time.sleep(2)

In [90]:
#Aller dans films 
tous_films = driver.find_element(By.XPATH, '//*[@title="Tous les films"]').click()
time.sleep(2)


In [6]:
# Assuming you already have the driver set up
# driver = webdriver.Chrome(executable_path='path/to/chromedriver')

# Corrigé par Adin -> Il faudrait créer une loop qui modifie le numéro ce qui marche
#wait = WebDriverWait(driver, 20)
#action = wait.until(EC.element_to_be_clickable((By.XPATH, '//a[@href="/films/genre-13027/"]')))
#action.click()

In [8]:
#Choisir les genres
#action =driver.find_element(By.XPATH,'//*[@class="filter-entity-select"]/div[1]/select/option[2]').click()

In [115]:
#Avoir feuille par feuille
films =[]
films_addedinfo=[]
links=[]
for each_fiche in driver.find_elements(By.CLASS_NAME,'mdl'):
    #titres
    titre=each_fiche.find_element(By.CLASS_NAME, "meta-title-link").text
    #date sortie
    date_sortie= each_fiche.find_element(By.CLASS_NAME, "date").text
    #durée
    duree =each_fiche.find_element(By.CLASS_NAME, "meta-body-info").text.split('/')[1]
    #element =each_fiche.execute_script("return document.evaluate('//div[@class=\"meta-body-item meta-body-info\"]/text()[3]', document).iterateNext().textContent;")
    #/span/following-sibling::text()[1]
    #element= each_fiche.find_element(By.CLASS_NAME, 'spacer')
    #duree =element.find_element(By.XPATH, "//text()[1]").text
    #duree= element.find_element(By.XPATH,("following-sibling::*[2]")).text #The following sibling2 makes that we get the date and not the /
    #//*[@id="content-layout"]/section[3]/div[2]/ul/li[1]/div/div[1]/div[1]/div[1]/text()[1]
    #Genres
    info =each_fiche.find_element(By.CLASS_NAME, "meta-body-info")
    genres= info.find_elements(By.CLASS_NAME, "xXx")
    #genre1
    genre_1 = genres[0].text
    #Genre 2 et 3
    if len(genres)>=2:
        genre_2= genres[1].text
        if len(genres)==3:
            genre_3=genres[2].text
        else:
            genre_3="NA"
    else:
        genre_2="NA"
        
    #réalisateur
    #realisateur= each_fiche.find_element(By.CLASS_NAME, "blue-link").text
    direction =each_fiche.find_element(By.CLASS_NAME, "meta-body-direction")
    realisateurs= direction.find_elements(By.CSS_SELECTOR, "a")
    #realisateur1
    real_1 = realisateurs[0].text
    #real 2
    if len(realisateurs)>=2:
        real_2= realisateurs[1].text
    else:
        real_2="NA"
    
    #Actors
    casting =each_fiche.find_element(By.CLASS_NAME, "meta-body-actor")
    actors= casting.find_elements(By.CSS_SELECTOR, "a")
    #actor1
    actor_1 = actors[0].text
    #real 2
    if len(actors)>=2:
        actor_2= actors[1].text
        if len(actors)==3:
            actor_3=actors[2].text
        else:
            actor_3="NA"
    else:
        actor_2="NA"
        
    #Ratings
    #First initiate them in NA if does not exist
    note_spectateur="NA"
    note_presse="NA"
    #Look for the notes
    rating = each_fiche.find_element(By.CLASS_NAME, "rating-holder")
    ratings=rating.find_elements(By.CLASS_NAME,'rating-item-content')
    #For the first one
    rating_type_1=ratings[0].find_element(By.CSS_SELECTOR,'a').text
    if(rating_type_1== "PRESSE"):
        note_presse= ratings[0].find_element(By.CLASS_NAME, 'stareval-note').text
    elif(rating_type_1== "SPECTATEURS"):
        note_spectateur= ratings[0].find_element(By.CLASS_NAME, 'stareval-note').text   
        
    #For the second one
    rating_type_2=ratings[1].find_element(By.CSS_SELECTOR,'a').text
    if(rating_type_2== "SPECTATEURS"):
        note_spectateur= ratings[1].find_element(By.CLASS_NAME, 'stareval-note').text 
    elif(rating_type_2== "PRESSE"): #sometimes it is not in the same order
        note_presse= ratings[1].find_element(By.CLASS_NAME, 'stareval-note').text
    
    #Now we are done with the information we could find on the first page => click on the link for each film and go see more info
    #Problem, if come back to main page, stale elements => work around it by first creating a vector with all the links and visiting at the end without coming back to the main page
    link= each_fiche.find_element(By.CLASS_NAME, "meta-title-link").get_attribute('href')
    links.append(link)
    

    films.append({
                "Titre": titre,
                "date_sortie" : date_sortie,
                "duree": duree,
                "genre1": genre_1,
                "genre2": genre_2,
                "genre3": genre_3,
                "réalisateur_1": real_1,
                "réalisateur_2": real_2,
                "actor_1": actor_1,
                "actor 2": actor_2,
                "actor 3": actor_3,
                "note_presse": note_presse,
                "note spectators": note_spectateur,
                "link":link,
            })
    
#Get info on second page
for link in links:
    driver.get(link)
    time.sleep(3)
    infos = driver.find_element(By.CLASS_NAME,'meta-body')
    #Scenaristes
    scenar = infos.find_element(By.XPATH,'div[3]')
    scenarists = scenar.find_elements(By.CSS_SELECTOR,'a')
    scenar_1 = scenarists[0].text
    
    #Technical informations
    techni= driver.find_element(By.CLASS_NAME, 'ovw-technical')
    #nationality
    natio=techni.find_element(By.CLASS_NAME, 'nationality').text
    
    #Palmares
    if techni.find_elements(By.XPATH,'div[3]/a'):
        distributor = techni.find_element(By.XPATH,'div[3]/a').text
    else: 
        distributor="NA"
    
    films_addedinfo.append({
                "Scenar": scenar_1,
                "nationality": natio,
                "distributor": distributor,
                "nominations": nominations,
            })
    
#dataframe
films_data = pd.DataFrame(films)
films_added= pd.DataFrame(films_addedinfo)
Movies = pd.concat([films_data, films_added], axis=1)
Movies

,Titre,date_sortie,duree,genre1,genre2,genre3,réalisateur_1,réalisateur_2,actor_1,actor 2,actor 3,note_presse,note spectators,link,Scenar,nationality,nominations
0,Les Gardiens de la Galaxie 3,3 mai 2023,2h 30min,Action,Fantastique,Science Fiction,James Gunn,NA,Chris Pratt,Zoe Saldana,Dave Bautista,"3,6","4,3",https://www.allocine.fr/film/fichefilm_gen_cfi...,James Gunn,U.S.A.,The Walt Disney Company France
1,Les Trois Mousquetaires: D'Artagnan,5 avril 2023,2h 01min,Aventure,Historique,NA,Martin Bourboulon,NA,François Civil,Vincent Cassel,Romain Duris,"3,7","4,0",https://www.allocine.fr/film/fichefilm_gen_cfi...,Alexandre De La Patellière,France,Pathé
2,Je verrai toujours vos visages,29 mars 2023,1h 58min,Drame,NA,NA,Jeanne Herry,NA,Adèle Exarchopoulos,Dali Benssalah,Leïla Bekhti,"4,1","4,4",https://www.allocine.fr/film/fichefilm_gen_cfi...,Jeanne Herry,France,StudioCanal
3,Jeanne du Barry,17 mai 2023,1h 56min,Drame,Historique,Romance,Maïwenn,NA,Maïwenn,Johnny Depp,Benjamin Lavernhe,"3,1","3,5",https://www.allocine.fr/film/fichefilm_gen_cfi...,Maïwenn,France,Le Pacte
4,"Super Mario Bros, le film",5 avril 2023,1h 32min,Animation,Comédie,Famille,Aaron Horvath,Michael Jelenic,Pierre Tessier,Chris Pratt,Jérémie Covillault,"3,1","4,2",https://www.allocine.fr/film/fichefilm_gen_cfi...,Matthew Fogel,U.S.A.,Universal Pictures International France
5,La Vie pour de vrai,19 avril 2023,1h 50min,Comédie,NA,Famille,Dany Boon,NA,Dany Boon,Charlotte Gainsbourg,Kad Merad,"2,8","2,7",https://www.allocine.fr/film/fichefilm_gen_cfi...,Dany Boon,France,Pathé
6,AKA,28 avril 2023,2h 00min,Action,Policier,Thriller,Morgan S. Dalibert,NA,Alban Lenoir,Eric Cantona,Thibault de Montalembert,"3,3","3,5",https://www.allocine.fr/film/fichefilm_gen_cfi...,Morgan S. Dalibert,France,0
7,Misanthrope,26 avril 2023,1h 58min,Policier,Thriller,NA,Damián Szifron,NA,Shailene Woodley,Ben Mendelsohn,Jovan Adepo,"3,7","3,8",https://www.allocine.fr/film/fichefilm_gen_cfi...,Damián Szifron,U.S.A.,Metropolitan FilmExport
8,Donjons & Dragons : L'Honneur des voleurs,12 avril 2023,2h 14min,Aventure,Fantastique,NA,Jonathan Goldstein (XII),John Francis Daley,Chris Pine,Michelle Rodriguez,Regé-Jean Page,"3,3","3,9",https://www.allocine.fr/film/fichefilm_gen_cfi...,Jonathan Goldstein (XII),U.S.A.,Paramount Pictures France
9,Burning Days,26 avril 2023,2h 08min,Drame,Thriller,NA,Emin Alper,NA,Selahattin Paşalı,Ekin Koç,Erol Babaoğlu,"3,7","3,9",https://www.allocine.fr/film/fichefilm_gen_cfi...,Emin Alper,Turquie,Memento Distribution
